In [1]:
import pandas as pd 
import dask as dd 
import numpy as np

# Step 1 

Merge movie titles with plots and ratings and actors 


In [2]:
df_title = pd.read_csv("../data/title.basics.tsv" , sep = "\t")
df_rat = pd.read_csv( "../data/title.ratings.tsv" , sep = "\t" )
df_wiki = pd.read_csv( "../data/wiki_movie_plots_deduped.csv" )


df_title = df_title[ [ "tconst" , "originalTitle" , "startYear"] ]
df_title.columns = [ "id_movie" , "title" , "year"]

df_wiki = df_wiki[ ["Title" , "Plot" , "Release Year"]]
df_wiki.columns = ["title" , "plot" , "year"]

#  df_rat 
df_rat = df_rat[ ["tconst" , "averageRating"]]  # .head()
df_rat.columns = ["id_movie" , "ratings"]


# treat the year as string 
df_title["year"] = df_title["year"].astype( str )
df_wiki["year"] = df_wiki["year"].astype( str )


df_wiki_title = df_title.merge( df_wiki , on =[ "title" , "year" ], how = "inner" )

df_wiki_rat = df_wiki_title.merge( df_rat , on = "id_movie" , how = "left" )



/home/afhuertas/data-science/env-sonnet/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_wiki_rat.head() #[ df_wiki_rat["title"] == "Alice in Wonderland"]

,id_movie,title,year,plot,ratings
0,tt0000399,Jack and the Beanstalk,1902,The earliest known adaptation of the classic f...,6.1
1,tt0000420,Alice in Wonderland,1903,"Alice follows a large white rabbit down a ""Rab...",6.3
2,tt0000439,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.3
3,tt0000487,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.1
4,tt0000488,The Land Beyond the Sunset,1912,Joe is an impoverished New York newsboy who li...,6.9


In [4]:
df_wiki_rat.shape

(34288, 5)

In [5]:
xx = list(df_wiki_rat["id_movie"].values)

# Process actors data 

In [6]:
df_principals = pd.read_csv("../data/title.principals.tsv" , sep = "\t") 

In [7]:
df_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [8]:
#df_principals["inx"] = df_principals["tconst"].apply( lambda x :  True if x in xx else False  )
df_principals = df_principals[df_principals['tconst'].isin(  xx  )]

In [9]:
df_principals.shape

(282886, 6)

In [10]:
df_principals = df_principals[ ["tconst" , "nconst"]] # id_pelicula , Id actor]

In [11]:
df_principals.columns = [ "id_movie" , "id_actor"] 

In [12]:
df_name = pd.read_csv("../data/name.basics.tsv" , sep = "\t") 

In [13]:
df_name = df_name[["nconst" , "primaryName"] ]
df_name.columns = [ "id_actor" , "name"]

In [14]:
df_name.head()

,id_actor,name
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman


In [15]:
df_merge = df_principals.merge( df_name , on = "id_actor" )

In [16]:
gp = df_merge.groupby( by = [ "id_movie"] )["name"].agg( [ lambda x : ",".join( [  name.replace(" " , "") for name in x ]  ) ])

In [17]:
gp = gp.reset_index()
gp.columns = ["id_movie" , "actors"]

In [18]:
gp.head()

,id_movie,actors
0,tt0000399,"JamesH.White,ThomasWhite,GeorgeS.Fleming,Edwin..."
1,tt0000420,"MayClark,CecilM.Hepworth,Blair,GeoffreyFaithfu..."
2,tt0000439,"EdwinS.Porter,DonaldGallaher,GilbertM.'Broncho..."
3,tt0000487,"EmilyLowry,SiegmundLubin,JackFrawley"
4,tt0000488,"BigelowCooper,MartinFuller,Mrs.WilliamBechtel,..."


In [19]:
df_wiki_rat.head()

,id_movie,title,year,plot,ratings
0,tt0000399,Jack and the Beanstalk,1902,The earliest known adaptation of the classic f...,6.1
1,tt0000420,Alice in Wonderland,1903,"Alice follows a large white rabbit down a ""Rab...",6.3
2,tt0000439,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.3
3,tt0000487,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.1
4,tt0000488,The Land Beyond the Sunset,1912,Joe is an impoverished New York newsboy who li...,6.9


In [20]:
df_final = df_wiki_rat.merge( gp  , on = "id_movie" , how = "left" )
df_final.shape

(34288, 6)

In [21]:
df_final.head()

,id_movie,title,year,plot,ratings,actors
0,tt0000399,Jack and the Beanstalk,1902,The earliest known adaptation of the classic f...,6.1,"JamesH.White,ThomasWhite,GeorgeS.Fleming,Edwin..."
1,tt0000420,Alice in Wonderland,1903,"Alice follows a large white rabbit down a ""Rab...",6.3,"MayClark,CecilM.Hepworth,Blair,GeoffreyFaithfu..."
2,tt0000439,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.3,"EdwinS.Porter,DonaldGallaher,GilbertM.'Broncho..."
3,tt0000487,The Great Train Robbery,1903,The film opens with two bandits breaking into ...,7.1,"EmilyLowry,SiegmundLubin,JackFrawley"
4,tt0000488,The Land Beyond the Sunset,1912,Joe is an impoverished New York newsboy who li...,6.9,"BigelowCooper,MartinFuller,Mrs.WilliamBechtel,..."
